# Part 3: Forecasting with a Prophet 📉

One piece of recent open software is facebook's `prophet`. This uses a model which is similar to a **generalised additive model** (GAM), a class of additive (linear) models with potentially non-linear components. 

The model is easily interpreted (as a sum of components) and simple to fit (parameters have a strightforward interpretation). But if the hypothesis of the parametric model are not respected, the fitted model may seriously underfit.

As you are beginning to see time series forecasting can sometimes be as much an art as a science, and we know it takes quite some practice to become a good artist. The authors of the software had this in mind when they wrote it. One of their aims is to provide software which can be used easily by those who have a little timeseries knowledge. Let's see how this goes...

**Resources**

- https://peerj.com/preprints/3190/
- https://www.youtube.com/watch?v=2XFro0nIHQM&list=PLjwX9KFWtvNnOc4HtsvaDf1XYG3O5bv5s&index=10
- https://www.youtube.com/watch?v=95-HMzxsghY
- https://www.youtube.com/watch?v=pOYAXv15r3A 

**Required librairies**
- [ ] prophet
- [ ] numpy
- [ ] pandas
- [ ] matplotlib
- [ ] seaborn
- [ ] pmdarima

To install prophet I advise you to install a completely clean virtual environment with conda (you will find the instructions to do this [here](https://docs.conda.io/projects/conda/en/4.6.1/user-guide/tasks/manage-environments.html)), with numpy, pandas, matplotlib, seaborn and pmdarima.

Then you can install prophet with: `conda install conda-forge::prophet`


In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pmdarima as pm
from prophet import Prophet
pd.plotting.register_matplotlib_converters()

# Example 1 - Airline data

Let's setup the data as we had done in the previous parts.

In [ ]:
from pmdarima.datasets import load_airpassengers

def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

START_DATE = '1949-01-01'
airline = load_airpassengers(as_series=True)

# There's no datetimeindex from the bundled dataset. So let's add one.
airline.index= pd.date_range(START_DATE, 
                             periods=len(airline), 
                             freq='MS')

airline_adj = airline / airline.index.days_in_month # Remove spurious features due to variations of months duration
train, test = ts_train_test_split(airline_adj, '1960-01-01') # Split the dataset into train and test set
train_log, test_log = np.log(train), np.log(test) # Apply the log transformation

## Forecasting with prophet

Fitting a basic model and making predictions is very simple with `prophet`. You simply need to create a Prophet object, and then call its `fit` method on the training data.

The training data is expected in this data frame format

In [ ]:
train_df = pd.DataFrame(train).reset_index().rename(columns = {0:'y','index':'ds'})
train_df.tail()

Fit model...

In [ ]:
model = Prophet()
model.fit(train_df)

Make predictions...

In [ ]:
future = model.make_future_dataframe(periods=12, 
                                     freq='MS', 
                                     include_history=True) 

# Create a dataframe with the prediction datetimes we want
future.head()

In [ ]:
forecast = model.predict(future)
y_pred = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].set_index('ds')

Plot predictions...

In [ ]:
fig = model.plot(forecast)
test.plot(style='.r')

Evaluating this baseline model error on test set...

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    '''
    MAPE

    Parameters:
    --------
    y_true -- np.array actual observations from time series
    y_pred -- the predictions to evaluate

    Returns:
    -------
    float, scalar value representing the MAPE (0-100)
    '''
    #y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mean_absolute_percentage_error(test.values , 
                               forecast['yhat'][-12:])

We see the Prophet at least beats the naive model that we defined in the last notebook (from the unlogged data). Not too bad, given the small number of lines of code! But looking at the plot it is not fitting very well. We should use a cross-validation scheme to find a better fitting model (before evaluating it on the test set!).

## Cross validation with Prophet

The `prophet` package also provides cross validation routines that perform adjustable rolling windows as we have seen previously in other cross validation routines specialized for time series data.

In [ ]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric

In [ ]:
365.25*4

In [ ]:
model = Prophet()
model.fit(train_df)
df_cv = cross_validation(model, 
                         initial='1461 days', 
                         period='365 days', 
                         horizon = '365 days')

In [ ]:
df_cv.head()

A function for performance metrics and plotting...

In [ ]:
df_p = performance_metrics(df_cv)

In [ ]:
df_p.head()

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='mape')
plt.show()

The plot above is the standard output plot of the CV from `prophet`. It is different to what we have looked at before. Each grey point represents a prediction, made on a particular month. Because we made predictions over several horizons each are plotted on the graph. At around horizon 30 for example we see each of the plots over the first month of each of the 6, 12 month validations. The blue line shows a rolling window of the mape scores 'averaged' over the cv predictions. More info [here](https://facebook.github.io/prophet/docs/diagnostics.html).

We are normally interested in the mean and std of the MAPE scores found at each window. This can be calculated from the dataframe returned from the `cross_validation` function above, using the function below. 

In [ ]:
def mape_performance_calc(df_cv):
    " function to replace prophet cv 'performance metrics' function"
    df_cv['mape'] = abs(df_cv['y'] - df_cv['yhat']) /df_cv['y'] # make column with individual abs normalised values
    results = df_cv.groupby(['cutoff']).mean() # finish mape calc for each of the folds (distinguised in 'cutoff' column of df_cv)
    results = results.describe() # sumarise folds information
    results = results['mape']# # return only mape column
    return(results)

mape_performance_calc(df_cv)

We find a MAPE of ~7.2% with std ~2%. This would have estimated the error we saw on the test set to a good degree.

## Hyper-parameters

The model plot we saw earlier did not fit very well to the train data. We can change the model assumptions (like hyper-parameters) to change how well it is fitting the train data.

`Prophet(
    growth='linear',
    changepoints=None,
    n_changepoints=25,
    changepoint_range=0.8,
    yearly_seasonality='auto',
    weekly_seasonality='auto',
    daily_seasonality='auto',
    holidays=None,
    seasonality_mode='additive',
    seasonality_prior_scale=10.0,
    holidays_prior_scale=10.0,
    changepoint_prior_scale=0.05,
    mcmc_samples=0,
    interval_width=0.8,
    uncertainty_samples=1000,
    stan_backend=None,
)`

**TODO**

* Try changing some of these. [This](https://medium.com/data-science/implementing-facebook-prophet-efficiently-c241305405a3) blog is very useful in explaining the parameters and what they do.
* Find at least one change which improves your model cross validation score

In [ ]:
#### your solution here 


# Optional exercise - Energy data forecasting ⚡

Forecasting energy demand is big business and very important in helping energy producers maintain the balance within the electrical grid. You can find forecasts of energy demands for France in realtime [here](https://www.rte-france.com/fr/eco2mix/eco2mix-consommation). The data was already downloaded in the `data/nats.csv` file.

Your mission now is to imagine a useful use-case for an enterprise and use `Prophet` to make forecasts on the energy data you have already seen (in data viz module) for France! Feel free to choose :
* the type of energy generation/consommation
* france or regional,
* frequency scale you wish to try!

Try and consider a forecast horizon that seems useful for the frequency at which you are making your predictions e.g. predicting the hourly forecast for 12 months propbably cannot be used in any meaningful way - and is probably not going be very easy to forecast accurately.

Consider that:
* Reducing the scale of the data, i.e. to regional
* Higher frequency predictions
* longer periods of forecast

...are likely to make it more difficult to make a good forecast.

Consider also that you must pick the most suitable cross validation procedure for your problem. Consider the initial, step and horizon you will use carefully.

Once you have made your model:
1. Compare it to a naive model!! If it does not beat this then try reducing your forecast horizon to find at what scale you can make useful predictions.
2. Try tuning the model in some way to improve performance (add holidays might be a good idea)
3. If you have time compare your model to an ARIMA model. Why might there be differences in perfomance between the two approachecs to forecasting?

(feel free to use a new notebook for this task!)